<a href="https://colab.research.google.com/github/LuisIZ/Lab1_AnimalSound_DeepLearning/blob/test_model/notebooks/eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laboratorio 1 - Animal Sound

## Librerías

In [3]:
# Para redes neuronales
import torch
import torch.nn as nn

# Para visualización de resultados
import matplotlib.pyplot as plt

# Para procesamiento de audio
import torchaudio
import librosa

# Para métricas
import sklearn

# Para manipulación de datos
import numpy as np
import pandas as pd

# Otros
import os, pathlib, random
from tqdm import tqdm
from google.colab import drive

## GPU

In [ ]:
!nvidia-smi

## Seed

In [4]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

## Análisis Exploratorio de Datos (EDA)

1. ¿Cuál es el objetivo del laboratorio?

Queremos desarrollar un modelo de clasificación multi-etiqueta que pueda identificar correctamente los animales presentes en las grabaciones de la selva amazónica.

2. ¿Qué tipo de datos tenemos en nuestro dataset?

Tenemos los archivos de audio en formato WAV en las carpetas `train/` y `test/`. Además, en la primera carpeta, tenemos un archivo CSV que tiene como primera columna el nombre del archivo o `filename` y el resto de columnas son los nombres de cada especie. En total son 43 columnas, la primera contendra strings mientras que el resto contendrá valores 0 o 1 que indican la ausencia o presencia de la especie en la grabación.

3. ¿Qué herramientas planeamos utilizar?

En principio, planeamos utilizar `PyTorch` para los modelos (para que utilicen redes neuronales), `Matplotlib` para crear los gráficos de nuestros resultados (ej. modelar el descenso de la gradiente o como va evolucionando los losses en la etapa de training y testing), `TorchAudio` o `Librosa` para analizar features del dataset (ej. Mel-spectogram, MFCC, etc.), `Sklearn` para obtener metricas (ej. f1 score, multilabel confusion matrix, ROC/PR curves etc.) y utilizar métodos de reducción de dimensionalidad (ej. PSA, TSNE, etc.), `Pandas` y `Numpy` para manipular la data y sacar alguna métricas estadísticas (ej. promedio, cuartiles, etc.).

4. ¿Qué restricciones tenemos?

Además del plazo de entrega que es de 1 semana, tenemos recursos computacionales limitados. Trabajaremos con Colab para aprovechar la GPU que nos brinda.

## Google Drive


Verificamos el acceso a Google Drive porque estamos desarrollando el laboratorio en VS Code con la extensión de Google Colab y desde la página web de este último para poder acceder sin problemas a la data en Google Drive que está como acceso directo (*symlink*) a una carpeta compartida.

In [ ]:
drive.mount("/content/drive", force_remount=True)

In [ ]:
MYDRIVE = pathlib.Path("/content/drive/MyDrive")

hits = list(MYDRIVE.rglob("Animal Sounds"))
if not hits:
    raise FileNotFoundError("No encuentro la carpeta 'Animal Sounds' dentro de MyDrive. Revisa que el acceso directo exista.")

DATA_ROOT = hits[0]          # la ruta “atajo”
REAL_ROOT = hits[0].resolve() # la ruta real (como symlink)

print("DATA_ROOT:", DATA_ROOT)
print("REAL_ROOT:", REAL_ROOT)

In [ ]:
TRAIN_7Z  = REAL_ROOT / "train.7z"
TEST_7Z   = REAL_ROOT / "test.7z"
TRAIN_CSV = REAL_ROOT / "train.csv"

for p in [TRAIN_7Z, TEST_7Z, TRAIN_CSV]:
    print(p, "=>", p.exists())

Habiendo encontrado las rutas de nuestros archivos:

- La carpeta comprimida con los datos de training (`train.7z`)
- La carpeta comprimida con los datos de testing (`test.7z`)
- El archivo csv con las multi-etiquetas de cada video, indicando que especie suena en el audio, 1, y cual no, 0 (`train.csv`)

En la siguiente sección, procedemos a descomprimir las carpetas, utilizando la herramienta `7z`, y revisar su contenido para poder realizar el Análisis Exploratorio de Datos (o *EDA* por sus siglas en Inglés).

## Dataset

Como trabajar con los datos directamente en Drive sería muy lento, procederemos a guardar el contenido de los archivos descomprimidos en `/content` que es el disco local temporal de la máquina de Colab que ofrece mayor velocidad de I/O. Al ser temporal, si se reinicia el entorno, es necesario repetir la descompresión.

In [ ]:
# Verificamos que haya suficiente espacio en /content
!df -h /content

In [ ]:
# Definimos rutas y creamos carpetas para guardar los datos
DRIVE_DATA = pathlib.Path("/content/drive/.shortcut-targets-by-id/1F5_zs2zy0oECJu6NSwXtSAsIL9HQJ3yf/Animal Sounds")
TRAIN_7Z = DRIVE_DATA / "train.7z"
TEST_7Z  = DRIVE_DATA / "test.7z"
TRAIN_CSV = DRIVE_DATA / "train.csv"

OUT_BASE = pathlib.Path("/content/data")
TRAIN_OUT = OUT_BASE / "train"
TEST_OUT  = OUT_BASE / "test"

TRAIN_OUT.mkdir(parents=True, exist_ok=True)
TEST_OUT.mkdir(parents=True, exist_ok=True)

print(TRAIN_7Z.exists(), TEST_7Z.exists(), TRAIN_CSV.exists())
print("Extracción a:", OUT_BASE)

In [ ]:
# Extraemos los datos
!7z x "/content/drive/.shortcut-targets-by-id/1F5_zs2zy0oECJu6NSwXtSAsIL9HQJ3yf/Animal Sounds/train.7z" -o"/content/data/train" -y
!7z x "/content/drive/.shortcut-targets-by-id/1F5_zs2zy0oECJu6NSwXtSAsIL9HQJ3yf/Animal Sounds/test.7z"  -o"/content/data/test"  -y

In [ ]:
# Verificamos si estan los audios
!find /content/data/train -maxdepth 2 -type f | head
!find /content/data/test  -maxdepth 2 -type f | head

In [ ]:
DATA_DIR = Path("/content/data")

def resolve_nested(split_dir: Path):
    nested = split_dir / split_dir.name
    return nested if nested.exists() else split_dir

TRAIN_DIR = resolve_nested(DATA_DIR / "train")
TEST_DIR  = resolve_nested(DATA_DIR / "test")

print("TRAIN_DIR:", TRAIN_DIR)
print("TEST_DIR :", TEST_DIR)
print("Ejemplo train existe:", TRAIN_DIR.exists())
print("Ejemplo test existe :", TEST_DIR.exists())

In [ ]:
df = pd.read_csv(TRAIN_CSV)
print(df.shape)
df.head()

In [ ]:
f0 = df["filename"].iloc[0]
print("Archivo:", f0)
print("Existe?:", (TRAIN_DIR / f0).exists())

In [ ]:
drive_root = pathlib.Path("/content/drive")

train7z = list(drive_root.rglob("train.7z"))
test7z  = list(drive_root.rglob("test.7z"))
csvs    = list(drive_root.rglob("train.csv"))

print("train.7z encontrados:", len(train7z))
print("test.7z encontrados:", len(test7z))
print("train.csv encontrados:", len(csvs))

# candidato: carpeta que tenga los 3 archivos
candidates = []
for p in train7z:
    folder = p.parent
    if (folder / "test.7z").exists() and (folder / "train.csv").exists():
        candidates.append(folder)

print("Carpetas candidatas:", len(candidates))
for c in candidates[:10]:
    print(" -", c)

DATASET_DIR = candidates[0]  # si sale >1, elegimos la que corresponde
print("Usando DATASET_DIR:", DATASET_DIR)

## EDA

In [ ]:
label_cols = df.columns[1:]   # las columnas de todas las especies en un audio
y = df[label_cols].values

print("Shape:", df.shape)
print("Num clases:", len(label_cols))
print("Audios únicos:", df["filename"].nunique())
print("Nulos totales:", df.isna().sum().sum())

In [ ]:
# ¿cuántas etiquetas por audio?
k = df[label_cols].sum(axis=1)
k.describe()

In [ ]:
# balance por clase
class_counts = df[label_cols].sum().sort_values(ascending=False)
class_counts.head(10), class_counts.tail(10)

In [ ]:
# porcentaje de precensia por clase
class_pct = (class_counts / len(df) * 100).sort_values(ascending=False)
class_pct.head(10)

## Feature Extraction

In [ ]:
# creamos carpeta para almacenar features
CACHE_DIR = Path("/content/drive/MyDrive/Lab1_cache")
CACHE_DIR.mkdir(parents=True, exist_ok=True)
print(CACHE_DIR)

In [ ]:
def extract_mfcc_stats(wav_path, sr=32000, n_mfcc=20):
    y, _ = librosa.load(wav_path, sr=sr, mono=True)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    feat = np.concatenate([mfcc.mean(axis=1), mfcc.std(axis=1)]).astype(np.float32)
    return feat

In [ ]:
n_mfcc = 20
feat_names = [f"mfcc_mean_{i}" for i in range(n_mfcc)] + [f"mfcc_std_{i}" for i in range(n_mfcc)]

In [ ]:
# OJO: Solo correrlo si necesitas volver a extraer y guardar los features
if H5_PATH.exists():
    os.remove(H5_PATH)

In [ ]:
# =========================
# CONFIG
# =========================
CACHE_DIR = Path("/content/drive/MyDrive/Lab1_cache")
CACHE_DIR.mkdir(parents=True, exist_ok=True)

H5_PATH = CACHE_DIR / "train_mfcc_stats.h5"
KEY = "train"
BATCH = 256  # ¿probar 256 o 512?

# para evitar el error de "string len limit"
MIN_ITEMSIZE = {"filename": 120}

# =========================
# (OPCIONAL) LIMPIAR H5 SI ESTÁ "MAL CREADO"
# =========================
# Si sale el error de itemsize, lo más limpio es borrar y regenerar
if H5_PATH.exists():
    print("H5 ya existe:", H5_PATH)
    print("Si este archivo fue creado antes SIN min_itemsize, podría fallar. "
          "Si vuelve a fallar, bórralo (os.remove) y reintenta.")
    # Descomentar si se quiere forzar recreación
    # os.remove(H5_PATH)

# =========================
# RECUPERAR PROCESADOS
# =========================
processed = set()
if H5_PATH.exists():
    try:
        with pd.HDFStore(H5_PATH, mode="r") as store:
            if f"/{KEY}" in store.keys():
                processed = set(store.select(KEY, columns=["filename"])["filename"].astype(str).tolist())
    except Exception as e:
        print("No se pudo leer el H5 para resume. Error:", repr(e))
        print("Recomendación: borrar el H5 y regenerar.")
        # os.remove(H5_PATH)
        processed = set()

print("Procesados previamente:", len(processed))

# =========================
# EXTRACCIÓN + GUARDADO POR BATCH
# =========================
rows = []
skipped_missing = 0
skipped_errors = 0
written = 0

for i in tqdm(range(len(df)), desc="Extrayendo MFCC"):
    fn = str(df.loc[i, "filename"])

    if fn in processed:
        continue

    wav_path = TRAIN_DIR / fn
    if not wav_path.exists():
        skipped_missing += 1
        continue

    try:
        feat = extract_mfcc_stats(str(wav_path), sr=32000, n_mfcc=n_mfcc)
    except Exception:
        skipped_errors += 1
        continue

    row = {"filename": fn}
    row.update({k: float(v) for k, v in zip(feat_names, feat)})

    for c in label_cols:
        row[c] = int(df.loc[i, c])

    rows.append(row)

    # flush por batch
    if len(rows) >= BATCH:
        out = pd.DataFrame(rows)

        # Guardar (con min_itemsize para filename)
        out.to_hdf(
            H5_PATH,
            key=KEY,
            mode="a",
            format="table",
            append=True,
            data_columns=["filename"],
            min_itemsize=MIN_ITEMSIZE,
            complib="blosc",
            complevel=5
        )

        written += len(out)
        rows = []

# flush final
if rows:
    out = pd.DataFrame(rows)
    out.to_hdf(
        H5_PATH,
        key=KEY,
        mode="a",
        format="table",
        append=True,
        data_columns=["filename"],
        min_itemsize=MIN_ITEMSIZE,
        complib="blosc",
        complevel=5
    )
    written += len(out)

print("Listo. Guardado en:", H5_PATH)
print("Escritos nuevos:", written)
print("Saltados (faltan archivos):", skipped_missing)
print("Saltados (errores lectura/audio):", skipped_errors)

In [ ]:
SAMPLE_N = 300
sample_files = df["filename"].sample(SAMPLE_N, random_state=42).tolist()
wav_paths = [str(TRAIN_DIR / f) for f in sample_files]

In [ ]:
H5_PATH = "/content/drive/MyDrive/Lab1_cache/train_mfcc_stats.h5"
df_feat = pd.read_hdf(H5_PATH, key="train")

print("Shape:", df_feat.shape)
print("Cols:", df_feat.columns[:10].tolist())
print("Filenames únicos:", df_feat["filename"].nunique())
print("Nulos totales:", df_feat.isna().sum().sum())

df_feat.head()

## Model

In [1]:
from google.colab import drive
drive.mount("/content/drive")  # sigue el link de autorización

Mounted at /content/drive


In [2]:
!ls -lah /content/drive/MyDrive
!ls -lah /content/drive/MyDrive/Lab1_cache

ls: '/content/drive/MyDrive/Animal Sounds': No such file or directory
total 3.0G
-rw------- 1 root root  181 Oct 16  2020 '1.01_1_Izaguirre Chiotti_ABP-M1.gslides'
-rw------- 1 root root 589K Apr  7  2022  1_13.xlsx
drwx------ 2 root root 4.0K Aug 15  2022  2022-2
-rw------- 1 root root 3.2M May 29  2025 '2024_1.9. Meta_XR_SDK_Building Blocks Multiplayer-1.docx.pdf'
-rw------- 1 root root  21K Mar  9  2024 '2928578 (1).jpg'
-rw------- 1 root root  21K Mar  9  2024  2928578.jpg
-rw------- 1 root root  43K Oct 30  2024  3c6fb1c2-4e8e-4fcd-bbcc-518351368d62.jpeg
-rw------- 1 root root 493K May 12  2022  4.0.wav
-rw------- 1 root root 481K May 12  2022  4.1.wav
-rw------- 1 root root 477K May 12  2022  4.2.wav
-rw------- 1 root root 483K May 12  2022  4.3.wav
-rw------- 1 root root 487K May 12  2022  4.4.wav
-rw------- 1 root root 495K May 12  2022  6.0.wav
-rw------- 1 root root 495K May 12  2022  6.1.wav
-rw------- 1 root root 505K May 12  2022  6.2.wav
-rw------- 1 root root 501K May 12

In [3]:
from pathlib import Path

cands = list(Path("/content/drive").rglob("train_mfcc_stats.h5"))
print("Encontrados:", len(cands))
for p in cands[:20]:
    print(p)

Encontrados: 1
/content/drive/MyDrive/Lab1_cache/train_mfcc_stats.h5


In [4]:
from pathlib import Path
import pandas as pd

CACHE_DIR = Path("/content/drive/MyDrive/Lab1_cache")
H5_PATH = CACHE_DIR / "train_mfcc_stats.h5"

assert H5_PATH.exists(), f"No encuentro: {H5_PATH}"

# mira qué keys tiene el HDF5 (para no fallar con el key)
with pd.HDFStore(H5_PATH, mode="r") as store:
    print("Keys:", store.keys())

df_feat = pd.read_hdf(H5_PATH, key="train")  # o el key que veas arriba
print(df_feat.shape)
df_feat.head()

Keys: ['/train']
(62191, 83)


,filename,mfcc_mean_0,mfcc_mean_1,mfcc_mean_2,mfcc_mean_3,mfcc_mean_4,mfcc_mean_5,mfcc_mean_6,mfcc_mean_7,mfcc_mean_8,...,SCINAS,LEPNOT,ADEMAR,BOAALM,PHYDIS,RHIORN,LEPFLA,SCIRIZ,DENELE,SCIALT
0,INCT20955_20190909_050000_0_3.wav,-189.088409,117.389236,-146.154938,55.875923,-7.179368,52.681614,-11.384820,22.849953,0.413092,...,0,0,0,0,0,0,0,0,0,0
1,INCT20955_20190909_050000_1_4.wav,-187.898651,116.883789,-146.542892,56.535400,-7.780136,51.700756,-9.963317,22.915176,-0.442919,...,0,0,0,0,0,0,0,0,0,0
2,INCT20955_20190909_050000_2_5.wav,-187.243317,109.707817,-139.818100,50.498848,-2.099809,47.686493,-6.434562,21.802526,0.812162,...,0,0,0,0,0,0,0,0,0,0
3,INCT20955_20190909_050000_3_6.wav,-185.078537,116.537796,-149.052811,55.316799,-7.776229,52.924919,-10.019419,23.378138,0.383413,...,0,0,0,0,0,0,0,0,0,0
4,INCT20955_20190909_050000_4_7.wav,-175.066849,107.470642,-139.312286,47.302567,-0.816672,48.910305,-7.007190,22.232889,2.580527,...,0,0,0,0,0,0,0,0,0,0


In [7]:
import pathlib
CACHE_DIR = pathlib.Path("/content/drive/MyDrive/Lab1_cache")
H5_PATH   = CACHE_DIR / "train_mfcc_stats.h5"
KEY       = "train"

df_feat = pd.read_hdf(H5_PATH, key=KEY)
print("Shape features:", df_feat.shape)
print(df_feat.head(2))

Shape features: (62191, 83)
                            filename  mfcc_mean_0  mfcc_mean_1  mfcc_mean_2  \
0  INCT20955_20190909_050000_0_3.wav  -189.088409   117.389236  -146.154938   
1  INCT20955_20190909_050000_1_4.wav  -187.898651   116.883789  -146.542892   

   mfcc_mean_3  mfcc_mean_4  mfcc_mean_5  mfcc_mean_6  mfcc_mean_7  \
0    55.875923    -7.179368    52.681614   -11.384820    22.849953   
1    56.535400    -7.780136    51.700756    -9.963317    22.915176   

   mfcc_mean_8  ...  SCINAS  LEPNOT  ADEMAR  BOAALM  PHYDIS  RHIORN  LEPFLA  \
0     0.413092  ...       0       0       0       0       0       0       0   
1    -0.442919  ...       0       0       0       0       0       0       0   

   SCIRIZ  DENELE  SCIALT  
0       0       0       0  
1       0       0       0  

[2 rows x 83 columns]


In [8]:
# OJO: esto asume que ya tenías label_cols y feat_names definidos.
# Si no, los reconstruimos con reglas simples:

# labels: todas las columnas menos filename y las de mfcc
non_label = {"filename"}
mfcc_cols = [c for c in df_feat.columns if c.startswith("mfcc_mean_") or c.startswith("mfcc_std_")]
label_cols = [c for c in df_feat.columns if c not in non_label and c not in set(mfcc_cols)]

feat_names = mfcc_cols  # tus X
print("Num feats:", len(feat_names))
print("Num labels:", len(label_cols))

Num feats: 40
Num labels: 42


In [10]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

X = df_feat[feat_names].values.astype(np.float32)
y = df_feat[label_cols].values.astype(np.float32)

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

train_dl = DataLoader(TensorDataset(torch.tensor(X_train), torch.tensor(y_train)),
                      batch_size=512, shuffle=True)
val_dl   = DataLoader(TensorDataset(torch.tensor(X_val), torch.tensor(y_val)),
                      batch_size=1024, shuffle=False)

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device, "cuda_available:", torch.cuda.is_available(),
      "device_count:", torch.cuda.device_count())

class MLP(nn.Module):
    def __init__(self, in_dim, out_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, out_dim),
        )
    def forward(self, x):
        return self.net(x)

model = MLP(X.shape[1], y.shape[1]).to(device)
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
crit = nn.BCEWithLogitsLoss()

def eval_model():
    model.eval()
    all_pred = []
    all_true = []
    with torch.no_grad():
        for xb, yb in val_dl:
            xb, yb = xb.to(device), yb.to(device)
            logits = model(xb)
            prob = torch.sigmoid(logits).cpu().numpy()
            all_pred.append(prob)
            all_true.append(yb.cpu().numpy())
    P = np.vstack(all_pred)
    T = np.vstack(all_true)

    # threshold simple (0.5). Luego puedes tunear.
    Yhat = (P >= 0.5).astype(int)
    micro = f1_score(T, Yhat, average="micro", zero_division=0)
    macro = f1_score(T, Yhat, average="macro", zero_division=0)
    return micro, macro

for epoch in range(1, 6):
    model.train()
    total_loss = 0.0
    for xb, yb in train_dl:
        xb, yb = xb.to(device), yb.to(device)
        opt.zero_grad()
        logits = model(xb)
        loss = crit(logits, yb)
        loss.backward()
        opt.step()
        total_loss += loss.item() * xb.size(0)

    micro, macro = eval_model()
    print(f"Epoch {epoch} | loss={total_loss/len(train_dl.dataset):.4f} | F1 micro={micro:.4f} | F1 macro={macro:.4f}")

Device: cuda cuda_available: True device_count: 1
Epoch 1 | loss=0.1851 | F1 micro=0.6153 | F1 macro=0.1966
Epoch 2 | loss=0.0739 | F1 micro=0.6727 | F1 macro=0.2732
Epoch 3 | loss=0.0633 | F1 micro=0.7051 | F1 macro=0.3195
Epoch 4 | loss=0.0574 | F1 micro=0.7292 | F1 macro=0.3476
Epoch 5 | loss=0.0534 | F1 micro=0.7524 | F1 macro=0.4081
